In [2]:
import pandas as pd

# Carregar o dataframe inicial
User = pd.read_excel('EVIO_history_01-02-2023_29-02-2024.xlsx')
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)', 'Nº cartão EVIO']]

# Remover valores de carregamento de energia inferiores a 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remover valores de duração de carregamento inferiores a 5 minutos
User = User[User['Duration (min)'] >= 5]

# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')

# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

# Mapeamento dos dias da semana para números
weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)

# Contar o número de observações por usuário
user_counts = User['Nº cartão EVIO'].value_counts()

# Filtrar usuários com mais de 100 observações
users_with_more_than_100_observations = user_counts[user_counts > 100].index

# Filtrar o DataFrame original para incluir apenas esses usuários
User = User[User['Nº cartão EVIO'].isin(users_with_more_than_100_observations)]

print(User)

              Start date           Stop date  Duration (min)  \
1    2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   
5    2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   
6    2024-02-29 08:53:00 2024-02-29 11:29:00         155.883   
10   2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
13   2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
...                  ...                 ...             ...   
1761 2023-02-03 08:12:00 2023-02-03 11:37:00         205.200   
1762 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
1763 2023-02-02 16:48:00 2023-02-02 19:25:00         156.800   
1770 2023-02-01 08:55:00 2023-02-01 11:38:00         162.833   
1771 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   

      Total Energy (kWh) Nº cartão EVIO  Weekday  
1                  19.95            144        4  
5                   9.14             46        4  
6                   8.89             76        4  
10                  8.60   

In [3]:
# Criar uma nova coluna contendo apenas a data (sem a hora)
User['Date'] = User['Start date'].dt.date

# Obter o intervalo de data
start_date = User['Date'].min()
end_date = User['Date'].max()

# Criar um novo DataFrame com todas as datas no intervalo
date_range = pd.date_range(start=start_date, end=end_date)

# Criar um DataFrame vazio para armazenar as datas sem carregamento
empty_dates = pd.DataFrame({'Date': date_range})

User['Date'] = pd.to_datetime(User['Date'], format='%Y/%m/%d')

# Mesclar os dois DataFrames usando apenas a coluna 'Date'
merged_df = pd.merge(empty_dates, User, on='Date', how='left')

# Preencher NaN e NaT por 0 em todo o DataFrame
merged_df = merged_df.fillna(0)


merged_df['Weekday'] = merged_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

merged_df['Weekday'] = merged_df['Weekday'].map(weekday_mapping)

print(merged_df)

          Date           Start date            Stop date  Duration (min)  \
0   2023-02-01  2023-02-01 08:55:00  2023-02-01 11:38:00         162.833   
1   2023-02-01  2023-02-01 08:50:00  2023-02-01 12:42:00         232.767   
2   2023-02-02  2023-02-02 16:48:00  2023-02-02 19:25:00         156.800   
3   2023-02-03  2023-02-03 08:12:00  2023-02-03 11:37:00         205.200   
4   2023-02-03  2023-02-03 08:08:00  2023-02-03 10:31:00         143.100   
..         ...                  ...                  ...             ...   
861 2024-02-28  2024-02-28 08:49:00  2024-02-28 12:20:00         211.183   
862 2024-02-28  2024-02-28 08:36:00  2024-02-28 13:27:00         290.983   
863 2024-02-29  2024-02-29 10:44:00  2024-02-29 11:24:00          39.483   
864 2024-02-29  2024-02-29 08:57:00  2024-02-29 10:11:00          74.783   
865 2024-02-29  2024-02-29 08:53:00  2024-02-29 11:29:00         155.883   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                  8.99             

In [4]:
# Convertendo as colunas Start date e Stop date para datetime, se necessário
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

print(merged_df['Start date'].dtype)
print(merged_df['Stop date'].dtype)
print(merged_df)

# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    merged_df[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0) apenas para as datas presentes no DataFrame original
for index, row in merged_df.iterrows():
    if not pd.isnull(row['Start date']) and not pd.isnull(row['Stop date']):
        start_hour = row['Start date'].hour
        stop_hour = row['Stop date'].hour
        for hour in range(8, 21):
            if hour >= start_hour and hour <= stop_hour:
                merged_df.at[index, f'Charging_{hour}h'] = 1

print(merged_df)

datetime64[ns]
datetime64[ns]
          Date          Start date           Stop date  Duration (min)  \
0   2023-02-01 2023-02-01 08:55:00 2023-02-01 11:38:00         162.833   
1   2023-02-01 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
2   2023-02-02 2023-02-02 16:48:00 2023-02-02 19:25:00         156.800   
3   2023-02-03 2023-02-03 08:12:00 2023-02-03 11:37:00         205.200   
4   2023-02-03 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
..         ...                 ...                 ...             ...   
861 2024-02-28 2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
862 2024-02-28 2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
863 2024-02-29 2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   
864 2024-02-29 2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   
865 2024-02-29 2024-02-29 08:53:00 2024-02-29 11:29:00         155.883   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  
0                  8.99       

In [5]:
# Converter colunas de data para datetime, tratando '0' como NaT
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Start date'] = pd.to_datetime(merged_df['Start date'], errors='coerce')
merged_df['Stop date'] = pd.to_datetime(merged_df['Stop date'], errors='coerce')

# Atualizar Start date e Stop date onde Total Energy é igual a zero
mask = merged_df['Total Energy (kWh)'] == 0
merged_df.loc[mask, 'Start date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)
merged_df.loc[mask, 'Stop date'] = merged_df.loc[mask, 'Date'] + pd.DateOffset(hours=0, minutes=0)

# Calcular a diferença em horas desde o último carregamento
hours_since_last_charge = []
previous_stop_date = None

for index, row in merged_df.iterrows():
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        hours_since_last_charge.append(0)
    else:
        if previous_stop_date is None:
            hours_since_last_charge.append(0)
        else:
            diff = (row['Start date'] - previous_stop_date).total_seconds() / 3600
            hours_since_last_charge.append(max(diff, 0))
    previous_stop_date = row['Stop date']

merged_df['Hours_since_last_charge'] = hours_since_last_charge

# Mostrar o DataFrame atualizado
print(merged_df)

          Date          Start date           Stop date  Duration (min)  \
0   2023-02-01 2023-02-01 08:55:00 2023-02-01 11:38:00         162.833   
1   2023-02-01 2023-02-01 08:50:00 2023-02-01 12:42:00         232.767   
2   2023-02-02 2023-02-02 16:48:00 2023-02-02 19:25:00         156.800   
3   2023-02-03 2023-02-03 08:12:00 2023-02-03 11:37:00         205.200   
4   2023-02-03 2023-02-03 08:08:00 2023-02-03 10:31:00         143.100   
..         ...                 ...                 ...             ...   
861 2024-02-28 2024-02-28 08:49:00 2024-02-28 12:20:00         211.183   
862 2024-02-28 2024-02-28 08:36:00 2024-02-28 13:27:00         290.983   
863 2024-02-29 2024-02-29 10:44:00 2024-02-29 11:24:00          39.483   
864 2024-02-29 2024-02-29 08:57:00 2024-02-29 10:11:00          74.783   
865 2024-02-29 2024-02-29 08:53:00 2024-02-29 11:29:00         155.883   

     Total Energy (kWh)  Nº cartão EVIO  Weekday  Charging_8h  Charging_9h  \
0                  8.99          

In [6]:
# Função para verificar se a hora x está dentro do intervalo de carregamento
def is_charging_at_hour(row, hour):
    if pd.isnull(row['Start date']) or pd.isnull(row['Stop date']):
        return 0
    start_hour = row['Start date'].hour + row['Start date'].minute / 60.0
    stop_hour = row['Stop date'].hour + row['Stop date'].minute / 60.0
    return 1 if start_hour <= hour < stop_hour else 0

# Lista de usuários únicos
users = merged_df['Nº cartão EVIO'].unique()
hours = range(8, 21)

# Criar um novo DataFrame para armazenar as informações agregadas
unique_dates = merged_df['Date'].unique()
new_columns = [f'Charging_{hour}h_user_{user}' for hour in hours for user in users]
new_df = pd.DataFrame(index=unique_dates, columns=new_columns)

# Preencher o novo DataFrame
for date in unique_dates:
    day_data = merged_df[merged_df['Date'] == date]
    for hour in hours:
        for user in users:
            user_data = day_data[day_data['Nº cartão EVIO'] == user]
            if not user_data.empty:
                charging_status = user_data.apply(lambda row: is_charging_at_hour(row, hour), axis=1).max()
                new_df.at[date, f'Charging_{hour}h_user_{user}'] = charging_status
            else:
                new_df.at[date, f'Charging_{hour}h_user_{user}'] = 0

# Substituir valores NaN por 0
new_df.fillna(0, inplace=True)

# Resetar o índice para ter a data como uma coluna
new_df.reset_index(inplace=True)
new_df.rename(columns={'index': 'Date'}, inplace=True)

new_df['Weekday'] = new_df['Date'].dt.day_name()

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

new_df['Weekday'] = new_df['Weekday'].map(weekday_mapping)

print(new_df)

# Inicializar a lista para armazenar os valores calculados
hours_since_last_charge_user_0 = []

# Inicializar a variável para armazenar a data e hora de parada anterior do usuário 0
previous_stop_date_user_0 = None

# Iterar sobre o DataFrame para calcular 'Hours_since_last_charge' para o usuário 0
for index, row in new_df.iterrows():
    if row['Charging_8h_user_0'] == 1:  # Considerar um exemplo de coluna específica do usuário 0
        if previous_stop_date_user_0 is None:
            hours_since_last_charge_user_0.append(0)
        else:
            # Calcular a diferença em horas desde o último carregamento do usuário 0
            diff = (row['Date'] - previous_stop_date_user_0).total_seconds() / 3600
            hours_since_last_charge_user_0.append(max(diff, 0))
        # Atualizar a data e hora de parada anterior para a próxima iteração
        previous_stop_date_user_0 = row['Date']
    else:
        if previous_stop_date_user_0 is None:
            hours_since_last_charge_user_0.append(0)
        else:
            diff = (row['Date'] - previous_stop_date_user_0).total_seconds() / 3600
            hours_since_last_charge_user_0.append(max(diff, 0))

# Adicionar a nova coluna ao DataFrame
new_df['Hours_since_last_charge_user_0'] = hours_since_last_charge_user_0
print(new_df['Hours_since_last_charge_user_0'])

          Date  Charging_8h_user_76  Charging_8h_user_29  Charging_8h_user_0  \
0   2023-02-01                    0                    0                   0   
1   2023-02-02                    0                    0                   0   
2   2023-02-03                    0                    0                   0   
3   2023-02-04                    0                    0                   0   
4   2023-02-05                    0                    0                   0   
..         ...                  ...                  ...                 ...   
389 2024-02-25                    0                    0                   0   
390 2024-02-26                    0                    0                   0   
391 2024-02-27                    0                    0                   0   
392 2024-02-28                    0                    0                   0   
393 2024-02-29                    0                    0                   0   

     Charging_8h_user_55  Charging_8h_u

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que 'new_df' foi criado conforme o código anterior e está disponível

# Definir o range de horas de interesse
hours = range(8, 21)

# Definir o usuário alvo para previsão
target_user = 0

# Preparar as colunas de entrada (X) e saída (y)
# Colunas para o y (Charging_{hour} do usuário alvo)
y_columns = [f'Charging_{hour}h_user_{target_user}' for hour in hours]

# Colunas para o X (Weekday e Charging_{hour} dos demais usuários)
X_columns = ['Weekday'] + [col for col in new_df.columns if col.startswith('Charging_') and not col.endswith(f'_user_{target_user}')]

# Separar X e y
X = new_df[X_columns].values
y = new_df[y_columns].values

#print(X,y)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

#print(X_test)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.9600778967867576
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
74  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
75  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
76  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
77  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
78  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  Charging_14h  Charg

In [35]:
# Função para concatenar as informações de horário e soma das colunas de carregamento
def concatenate_info(group):
    data = {
        'Start times': ', '.join(group['Start date'].astype(str)),
        'Stop times': ', '.join(group['Stop date'].astype(str)),
        'Durations (min)': ', '.join(group['Duration (min)'].astype(str)),
        'Total Energies (kwh)': ', '.join(group['Total Energy (kWh)'].astype(str)),
        'Nº cartões EVIO': ', '.join(group['Nº cartão EVIO'].astype(str)),
        'Weekday': group['Weekday'].iloc[0]
    }
    for hour in hours:
        data[f'Charging_{hour}h'] = group[f'Charging_{hour}h'].sum()
    return pd.Series(data)

# Agrupar por data e aplicar a função de concatenação
User_combined = merged_df.groupby('Date').apply(concatenate_info).reset_index()

# Calcular a diferença em horas desde o último carregamento
User_combined['Date'] = pd.to_datetime(User_combined['Date'])

# Inicializar a coluna de horas desde o último carregamento
User_combined['Hours_since_last_charge'] = 0

previous_stop_date = None

for index, row in User_combined.iterrows():
    current_start_dates = pd.to_datetime(row['Start times'].split(', '))
    if previous_stop_date is None:
        User_combined.at[index, 'Hours_since_last_charge'] = 0
    else:
        min_diff = min((current_start_date - previous_stop_date).total_seconds() / 3600 for current_start_date in current_start_dates)
        User_combined.at[index, 'Hours_since_last_charge'] = max(min_diff, 0)
    previous_stop_date = pd.to_datetime(row['Stop times'].split(', ')).max()

# Mostrar o DataFrame atualizado
print(User_combined)

          Date                                        Start times  \
0   2023-02-01           2023-02-01 08:55:00, 2023-02-01 08:50:00   
1   2023-02-02                                2023-02-02 16:48:00   
2   2023-02-03           2023-02-03 08:12:00, 2023-02-03 08:08:00   
3   2023-02-04                                         2023-02-04   
4   2023-02-05                                         2023-02-05   
..         ...                                                ...   
389 2024-02-25                                2024-02-25 10:20:00   
390 2024-02-26                                         2024-02-26   
391 2024-02-27  2024-02-27 16:35:00, 2024-02-27 08:50:00, 2024...   
392 2024-02-28           2024-02-28 08:49:00, 2024-02-28 08:36:00   
393 2024-02-29  2024-02-29 10:44:00, 2024-02-29 08:57:00, 2024...   

                                            Stop times  \
0             2023-02-01 11:38:00, 2023-02-01 12:42:00   
1                                  2023-02-02 19:25:00 

In [36]:
# Filtra os dados apenas para as colunas necessárias
columns_needed = ['Date', 'Nº cartão EVIO', 'Weekday'] + [f'Charging_{hour}h' for hour in range(24)]
data = data[columns_needed]

# Definindo o ID do usuário alvo e os IDs dos outros usuários
user_id = 29
other_users_ids = data['Nº cartão EVIO'].unique().tolist()
other_users_ids.remove(user_id)

# Função para preparar os dados
def prepare_data(data, user_id, other_users_ids):
    X, y = [], []
    for date in data['Date'].unique():
        # Dados do usuário alvo
        user_data = data[(data['Date'] == date) & (data['Nº cartão EVIO'] == user_id)]
        if user_data.empty:
            continue
        user_hours = user_data[[f'Charging_{hour}h' for hour in range(24)]].values.flatten()
        y.append(user_hours)
        
        # Dados dos outros usuários
        other_users_data = data[(data['Date'] == date) & (data['Nº cartão EVIO'].isin(other_users_ids))]
        if other_users_data.empty:
            continue
        other_users_hours = other_users_data[[f'Charging_{hour}h' for hour in range(24)]].values.flatten()
        
        # Adiciona o weekday
        weekday = user_data['Weekday'].values[0]
        features = np.append(other_users_hours, weekday)
        
        X.append(features)
    
    return np.array(X), np.array(y)

# Preparando os dados
X, y = prepare_data(data, user_id, other_users_ids)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazendo previsões para o conjunto de teste
y_pred = model.predict(X_test)

# Avaliando o modelo
print("Previsões:", y_pred)
print("Verdadeiro:", y_test)

NameError: name 'data' is not defined

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday', 'Hours_since_last_charge']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.7793987621573828
Previsões:
      Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0        Carregar      Carregar      Carregar      Carregar      Carregar   
1        Carregar      Carregar      Carregar      Carregar      Carregar   
2    Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3        Carregar      Carregar      Carregar      Carregar      Carregar   
4        Carregar      Carregar      Carregar      Carregar      Carregar   
..            ...           ...           ...           ...           ...   
169  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
170      Carregar      Carregar      Carregar      Carregar  Não Carregar   
171  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
172      Carregar      Carregar      Carregar      Carregar  Não Carregar   
173      Carregar      Carregar      Carregar      Carregar  Não Carregar   

     Charging_13h  Chargi

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Definir o range de horas de interesse
#hours = range(8, 21)

# Preparar a matriz de entrada (X) e saída (y)
X = merged_df[['Weekday']].values
y = merged_df[[f'Charging_{hour}h' for hour in hours]].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo
accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.7948717948717948
Previsões:
      Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0    Não Carregar      Carregar      Carregar      Carregar      Carregar   
1    Não Carregar      Carregar      Carregar      Carregar      Carregar   
2    Não Carregar      Carregar      Carregar      Carregar  Não Carregar   
3    Não Carregar      Carregar      Carregar      Carregar  Não Carregar   
4    Não Carregar      Carregar      Carregar      Carregar  Não Carregar   
..            ...           ...           ...           ...           ...   
169  Não Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
170  Não Carregar      Carregar      Carregar  Não Carregar  Não Carregar   
171  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
172  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
173  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

     Charging_13h  Chargi

In [29]:
hours = range(8, 21)

# Modelagem por usuário
results = []

for user in users_with_more_than_100_observations:
    user_data = merged_df[merged_df['Nº cartão EVIO'] == user]

    # Preparar a matriz de entrada (X) e saída (y)
    X = user_data[['Weekday']].values
    y = user_data[[f'Charging_{hour}h' for hour in hours]].values

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Treinar o modelo de regressão logística multinomial
    model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
    model.fit(X_train, y_train)

    # Fazer previsões para o conjunto de teste
    predictions = model.predict(X_test)

    # Avaliar a precisão do modelo
    accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
    print(f"Acurácia do modelo para o usuário {user}:", accuracy)

    # Criar uma tabela de previsões com 0s e 1s
    predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
    predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

    # Exibir as previsões
    print(f"Previsões para o usuário {user}:")
    print(predictions_table)

    # Calcular a matriz de confusão geral
    overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

    # Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
    overall_true_positives = overall_confusion_matrix[1, 1]
    overall_true_negatives = overall_confusion_matrix[0, 0]

    # Calcular o total de exemplos positivos e negativos
    total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
    total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

    # Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
    percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
    percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

    # Imprimir os resultados
    print(f'Percentagem de True Positives para o usuário {user}: {percentage_true_positives:.2f}%')
    print(f'Percentagem de True Negatives para o usuário {user}: {percentage_true_negatives:.2f}%')

    results.append({
        'user': user,
        'accuracy': accuracy,
        'true_positives': percentage_true_positives,
        'true_negatives': percentage_true_negatives
    })

# Exibir os resultados para todos os usuários
results_df = pd.DataFrame(results)
print(results_df)

Acurácia do modelo para o usuário 55: 0.5811965811965812
Previsões para o usuário 55:
     Charging_8h Charging_9h Charging_10h  Charging_11h Charging_12h  \
0       Carregar    Carregar     Carregar      Carregar     Carregar   
1       Carregar    Carregar     Carregar      Carregar     Carregar   
2       Carregar    Carregar     Carregar      Carregar     Carregar   
3       Carregar    Carregar     Carregar      Carregar     Carregar   
4       Carregar    Carregar     Carregar  Não Carregar     Carregar   
5       Carregar    Carregar     Carregar  Não Carregar     Carregar   
6       Carregar    Carregar     Carregar      Carregar     Carregar   
7       Carregar    Carregar     Carregar      Carregar     Carregar   
8   Não Carregar    Carregar     Carregar      Carregar     Carregar   
9       Carregar    Carregar     Carregar  Não Carregar     Carregar   
10      Carregar    Carregar     Carregar      Carregar     Carregar   
11      Carregar    Carregar     Carregar      Car

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [27]:
hours = range(8, 19)

# Modelagem por usuário
results = []

for user in users_with_more_than_100_observations:
    user_data = merged_df[merged_df['Nº cartão EVIO'] == user]

    # Preparar a matriz de entrada (X) e saída (y)
    X = user_data[['Weekday']].values
    y = user_data[[f'Charging_{hour}h' for hour in hours]].values

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Treinar o modelo de regressão logística multinomial
    model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
    model.fit(X_train, y_train)

    # Fazer previsões para o conjunto de teste
    predictions = model.predict(X_test)

    # Avaliar a precisão do modelo
    accuracy = accuracy_score(y_test.ravel(), predictions.ravel())
    print(f"Acurácia do modelo para o usuário {user}:", accuracy)

    # Criar uma tabela de previsões com 0s e 1s
    predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in hours])
    predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

    # Exibir as previsões
    print(f"Previsões para o usuário {user}:")
    print(predictions_table)

    # Calcular a matriz de confusão geral
    overall_confusion_matrix = confusion_matrix(y_test.ravel(), predictions.ravel())

    # Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
    overall_true_positives = overall_confusion_matrix[1, 1]
    overall_true_negatives = overall_confusion_matrix[0, 0]

    # Calcular o total de exemplos positivos e negativos
    total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
    total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

    # Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
    percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
    percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

    # Imprimir os resultados
    print(f'Percentagem de True Positives para o usuário {user}: {percentage_true_positives:.2f}%')
    print(f'Percentagem de True Negatives para o usuário {user}: {percentage_true_negatives:.2f}%')

    results.append({
        'user': user,
        'accuracy': accuracy,
        'true_positives': percentage_true_positives,
        'true_negatives': percentage_true_negatives
    })

# Exibir os resultados para todos os usuários
results_df = pd.DataFrame(results)
print(results_df)

Acurácia do modelo para o usuário 55: 0.5126262626262627
Previsões para o usuário 55:
     Charging_8h Charging_9h Charging_10h  Charging_11h Charging_12h  \
0       Carregar    Carregar     Carregar      Carregar     Carregar   
1       Carregar    Carregar     Carregar      Carregar     Carregar   
2       Carregar    Carregar     Carregar      Carregar     Carregar   
3       Carregar    Carregar     Carregar      Carregar     Carregar   
4       Carregar    Carregar     Carregar  Não Carregar     Carregar   
5       Carregar    Carregar     Carregar  Não Carregar     Carregar   
6       Carregar    Carregar     Carregar      Carregar     Carregar   
7       Carregar    Carregar     Carregar      Carregar     Carregar   
8   Não Carregar    Carregar     Carregar      Carregar     Carregar   
9       Carregar    Carregar     Carregar  Não Carregar     Carregar   
10      Carregar    Carregar     Carregar      Carregar     Carregar   
11      Carregar    Carregar     Carregar      Car